### Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
engine = create_engine(f"mysql://{root}:{password}@{localhost}/{database}")
inspector = inspect(engine)

In [4]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
session = Session(engine)

In [5]:
Base = declarative_base()

### Extract CSVs into DataFrames

In [6]:
athlete_info = pd.read_csv("olympic history/athlete_events.csv")
country_info = pd.read_csv("olympic history/noc_regions.csv")
olympic_df = pd.merge(athlete_info, country_info, how="left", on = "NOC")
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


In [7]:
olympic_df["medal_won"] = (olympic_df["Medal"] != "NaN")

poverty_df = pd.read_csv("global poverty/MPI_national.csv")
poverty_df.head()

,ISO,Country,MPI Urban,Headcount Ratio Urban,Intensity of Deprivation Urban,MPI Rural,Headcount Ratio Rural,Intensity of Deprivation Rural
0,KAZ,Kazakhstan,0.000,0.0,33.3,0.000,0.09,33.3
1,SRB,Serbia,0.000,0.1,41.4,0.002,0.50,40.3
2,KGZ,Kyrgyzstan,0.000,0.1,40.2,0.003,0.70,37.1
3,TUN,Tunisia,0.000,0.1,35.6,0.012,3.18,38.7
4,ARM,Armenia,0.001,0.2,33.3,0.001,0.39,36.9


In [8]:
olympic_df["medal_won"] = (olympic_df["Medal"] == "Gold")
olympic_df["medal_won"].value_counts()

False    257744
True      13372
Name: medal_won, dtype: int64

## Transform DataFrame

In [9]:
# Rename column "region"
olympic_df = olympic_df.rename(columns = {"region":"Country"})
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country,notes,medal_won
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,False
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,False
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,True
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,False


### Table 1 (Number of medals won per country)

In [10]:
medals_df = olympic_df['Medal'].dropna(how = 'any')
medals_df.head()

3       Gold
37    Bronze
38    Bronze
40    Bronze
41    Bronze
Name: Medal, dtype: object

In [11]:
# Aggregate data by region 
by_country = olympic_df.groupby("Country")
medal_count = by_country["Medal"].count()
medal_count

Country
Afghanistan                  2
Albania                      0
Algeria                     17
American Samoa               0
Andorra                      0
Angola                       0
Antigua                      0
Argentina                  274
Armenia                     16
Aruba                        0
Australia                 1349
Austria                    450
Azerbaijan                  44
Bahamas                     40
Bahrain                      3
Bangladesh                   0
Barbados                     1
Belarus                    139
Belgium                    468
Belize                       0
Benin                        0
Bermuda                      1
Bhutan                       0
Boliva                       0
Bosnia and Herzegovina       0
Botswana                     1
Brazil                     475
British Virgin Islands       0
Brunei                       0
Bulgaria                   342
                          ... 
Suriname                     2


In [12]:
#medal_count.sum()
agg_olympic_df = pd.DataFrame({"Number of Medals Won": medal_count})


# Merge dataframes 
combined_df = pd.merge(agg_olympic_df, poverty_df, how = "inner", on = "Country")
combined_df.head(15)

,Country,Number of Medals Won,ISO,MPI Urban,Headcount Ratio Urban,Intensity of Deprivation Urban,MPI Rural,Headcount Ratio Rural,Intensity of Deprivation Rural
0,Afghanistan,2,AFG,0.132,28.8,45.8,0.347,64.66,53.6
1,Albania,0,ALB,0.003,0.8,37.7,0.007,1.87,37.7
2,Algeria,17,DZA,0.003,0.7,36.9,0.011,2.70,40.0
3,Armenia,16,ARM,0.001,0.2,33.3,0.001,0.39,36.9
4,Azerbaijan,44,AZE,0.009,2.5,37.1,0.035,8.80,40.2
5,Bangladesh,0,BGD,0.100,22.5,44.7,0.231,48.21,47.9
6,Barbados,1,BRB,0.004,1.1,34.2,0.002,0.57,34.3
7,Belize,0,BLZ,0.006,1.6,37.9,0.028,6.98,39.9
8,Benin,0,BEN,0.192,41.0,46.8,0.386,76.90,50.2
9,Bhutan,0,BTN,0.019,4.7,39.5,0.159,35.98,44.2


In [13]:
#Extract only relevant columns
medals_df = combined_df[["Country", "Number of Medals Won", "MPI Urban", "MPI Rural", "Headcount Ratio Urban", "Headcount Ratio Rural"]]
medals_df.head(10)

,Country,Number of Medals Won,MPI Urban,MPI Rural,Headcount Ratio Urban,Headcount Ratio Rural
0,Afghanistan,2,0.132,0.347,28.8,64.66
1,Albania,0,0.003,0.007,0.8,1.87
2,Algeria,17,0.003,0.011,0.7,2.70
3,Armenia,16,0.001,0.001,0.2,0.39
4,Azerbaijan,44,0.009,0.035,2.5,8.80
5,Bangladesh,0,0.100,0.231,22.5,48.21
6,Barbados,1,0.004,0.002,1.1,0.57
7,Belize,0,0.006,0.028,1.6,6.98
8,Benin,0,0.192,0.386,41.0,76.90
9,Bhutan,0,0.019,0.159,4.7,35.98


### Table 2 (Number of gold medals won per country)

In [14]:
gold_medals = olympic_df[olympic_df["Medal"] == "Gold"]
gold_by_country = gold_medals.groupby("Country")

gold_medal_count = gold_by_country["Medal"].count()
agg_olympic_df2 = pd.DataFrame({"Number of Gold Medals Won": gold_medal_count})
#agg_olympic_df2.head()

In [15]:
combined_df2 = pd.merge(agg_olympic_df2, poverty_df, how = "inner", on = "Country")
gold_medals_df = combined_df2[["Country", "Number of Gold Medals Won", "MPI Urban", "MPI Rural", "Headcount Ratio Urban", "Headcount Ratio Rural"]]
gold_medals_df.head(15)

,Country,Number of Gold Medals Won,MPI Urban,MPI Rural,Headcount Ratio Urban,Headcount Ratio Rural
0,Algeria,5,0.003,0.011,0.7,2.70
1,Armenia,2,0.001,0.001,0.2,0.39
2,Azerbaijan,7,0.009,0.035,2.5,8.80
3,Brazil,109,0.016,0.054,4.1,11.96
4,Burundi,1,0.209,0.481,43.1,84.91
5,Cameroon,20,0.091,0.393,19.9,70.18
6,China,351,0.007,0.028,1.9,6.74
7,Colombia,5,0.009,0.061,2.3,14.44
8,Dominican Republic,3,0.029,0.048,7.9,11.69
9,Ecuador,1,0.005,0.030,1.5,7.68


### Table 3 (Number of medals won by men vs. women) 

In [40]:
by_gender = olympic_df.groupby("Sex")
medals_won = by_gender["Medal"].count()
agg_olympic_df4 = pd.DataFrame({"Number of Medals Won": medals_won})
agg_olympic_df4 = agg_olympic_df4.reset_index()
agg_olympic_df4.head()

,Sex,Number of Medals Won
0,F,11253
1,M,28530


### Table 4 (Number of medals won by men vs. women/country) 

In [38]:
by_country = olympic_df.groupby(["Country", "Sex"])
medals_won = by_country["Medal"].count()
agg_olympic_df3 = pd.DataFrame({"Number of Medals Won": medals_won})
agg_olympic_df3 = agg_olympic_df3.reset_index()
agg_olympic_df3.head()

,Country,Sex,Number of Medals Won
0,Afghanistan,F,0
1,Afghanistan,M,2
2,Albania,F,0
3,Albania,M,0
4,Algeria,F,3


### Create database connection

In [37]:
# done at the start of the code

,Country,Sex,Number of Medals Won
0,Afghanistan,F,0
1,Afghanistan,M,2
2,Albania,F,0
3,Albania,M,0
4,Algeria,F,3


### Load DataFrames into database

In [19]:
medals_df.to_sql('medals_and_poverty', con=engine, if_exists = 'replace')
#medals_df.head()

In [20]:
gold_medals_df.to_sql('gold_medals_and_poverty', con=engine, if_exists = 'replace')

In [41]:
agg_olympic_df4.to_sql('medals_won_and_sex', con=engine, if_exists = 'replace')

In [39]:
agg_olympic_df3.to_sql('medals_won_and_sex_by_country', con=engine, if_exists = 'replace')